In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, round

# Inicializar Spark con el conector de Cassandra
spark = SparkSession.builder \
    .appName("Pipeline_BigData_Fase3") \
    .config("spark.cassandra.connection.host", "cassandra_db") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .getOrCreate()

# Tarea 3.1: Lectura de la tabla 'ventas_crudas'
# Especificamos el keyspace y la tabla para la lectura paralela 
df_crudo = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="ventas_crudas", keyspace="proyecto_bigdata") \
    .load()

print("✅ Datos leídos exitosamente desde Cassandra.")
df_crudo.printSchema()

Py4JJavaError: An error occurred while calling o32.load.
: java.lang.NoClassDefFoundError: com/datastax/spark/connector/util/Logging
	at java.base/java.lang.ClassLoader.defineClass1(Native Method)
	at java.base/java.lang.ClassLoader.defineClass(ClassLoader.java:1017)
	at java.base/java.security.SecureClassLoader.defineClass(SecureClassLoader.java:150)
	at java.base/jdk.internal.loader.BuiltinClassLoader.defineClass(BuiltinClassLoader.java:862)
	at java.base/jdk.internal.loader.BuiltinClassLoader.findClassOnClassPathOrNull(BuiltinClassLoader.java:760)
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClassOrNull(BuiltinClassLoader.java:681)
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:639)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.cassandra.DefaultSource.getTable(DefaultSource.scala:55)
	at org.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, round

# 1. Inicializar Spark con la descarga automática del conector
# Nota: La versión 3.5.0 es compatible con Spark 3.x
spark = SparkSession.builder \
    .appName("Pipeline_BigData_Fase3") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .config("spark.cassandra.connection.host", "cassandra_db") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .getOrCreate()

# 2. Tarea 3.1: Lectura de la tabla 'ventas_crudas'
try:
    df_crudo = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(table="ventas_crudas", keyspace="proyecto_bigdata") \
        .load()
    print("✅ Datos leídos exitosamente desde Cassandra.")
    df_crudo.printSchema()
except Exception as e:
    print(f"❌ Error al leer de Cassandra: {e}")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, round

# Detener sesiones previas para evitar conflictos de configuración
try:
    spark.stop()
except:
    pass

# Inicializar con el paquete del conector de Datastax
spark = SparkSession.builder \
    .appName("Pipeline_BigData_Fase3") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .config("spark.cassandra.connection.host", "cassandra_db") \
    .getOrCreate()

print("✅ Sesión de Spark lista con el conector de Cassandra cargado.")

In [ ]:
# Tarea 3.1: Lectura distribuida
df_crudo = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="ventas_crudas", keyspace="proyecto_bigdata") \
    .load()

# Tarea 3.3: Limpieza (Filtro de montos positivos)
df_limpio = df_crudo.filter(col("monto_total") > 0) [cite: 23]

# Tarea 3.2: Lógica de Transformación
# Agrupar por fecha y categoría, sumar montos y contar transacciones
df_resumen = df_limpio.groupBy("fecha_venta", "categoria") \
    .agg(
        round(sum("monto_total"), 2).alias("ventas_totales"),
        count("id_venta").alias("cantidad_transacciones")
    ) [cite: 23]

print("📊 Vista previa de los datos transformados:")
df_resumen.show(10)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, round

# Detener cualquier sesión previa por seguridad
try:
    spark.stop()
except:
    pass

# Inicializar sesión cargando el conector de Datastax desde Maven
spark = SparkSession.builder \
    .appName("Pipeline_BigData_Fase3") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .config("spark.cassandra.connection.host", "cassandra_db") \
    .getOrCreate()

print("✅ Sesión de Spark reiniciada y conector cargado.")

In [ ]:
# 3.1. Lectura distribuida
df_crudo = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="ventas_crudas", keyspace="proyecto_bigdata") \
    .load()

# 3.3. Limpieza de datos (Requisito opcional recomendado) 
df_limpio = df_crudo.filter(col("monto_total") > 0)

# 3.2. Lógica de Transformación 
df_resumen = df_limpio.groupBy("fecha_venta", "categoria") \
    .agg(
        round(sum("monto_total"), 2).alias("ventas_totales"),
        count("id_venta").alias("cantidad_transacciones")
    )

# Mostrar resultados para la validación
df_resumen.show(10)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, round

# Detener cualquier rastro de sesiones previas
try:
    spark.stop()
except:
    pass

# Crear la sesión inyectando el conector necesario
spark = SparkSession.builder \
    .appName("Pipeline_BigData_Fase3") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.0") \
    .config("spark.cassandra.connection.host", "cassandra_db") \
    .getOrCreate()

print("✅ Sesión de Spark lista y conector descargado.")

In [ ]:
# 3.1 Lectura distribuida usando la Partition Key (fecha_venta) [cite: 23]
df_crudo = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="ventas_crudas", keyspace="proyecto_bigdata") \
    .load()

# 3.3 Limpieza: Quitar ventas con monto negativo o nulo [cite: 23]
df_limpio = df_crudo.filter(col("monto_total") > 0)

# 3.2 Lógica de Transformación: Agrupación y Cálculos [cite: 23]
df_resumen = df_limpio.groupBy("fecha_venta", "categoria") \
    .agg(
        round(sum("monto_total"), 2).alias("ventas_totales"),
        count("id_venta").alias("cantidad_transacciones")
    )

print("📊 ¡Éxito! Aquí tienes el resumen analítico:")
df_resumen.show(10)